In [ ]:
import pandas as pd
import numpy as np
import scipy as sci

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

import os
import sys
import glob
import re

from pathlib import Path

from itertools import compress

import pickle

In [ ]:
# Get the directory of the running notebook
notebook_dir = Path(os.getcwd())
project_root = notebook_dir.parent  # Adjust based on your project structure

sys.path.append(str(project_root))

In [ ]:
from Importfunctions.import_specs import * # allgemeine Datensatzverarbeitungsfunktionen
from Variables.Classes import *
from Variables.constants import *
from Darstellungen.axis_config import *
from Postprocessing.data_sanitizing import spec_selec_dataset, entferne_nbekannt
from Darstellungen.legend import *

In [ ]:
# erhöhe die Seaborn-Auflösung
sns.set_theme(rc={"figure.dpi": 600})

# Datenimport

## Lade die Datensatz-Objekte mit den Spektren und Befundlisten

In [ ]:
os.chdir("path/to/pickled/data")
with open('df_pickled.pickle','rb') as f:
    ds = pickle.load(f)

## Lade die Kategorienliste

In [ ]:
os.chdir("path/to/pickled/data")
with open('kategorien.pkl', 'rb') as f:
    kategorienliste = pickle.load(f)

# Erweitere die Dataset-Klasse

In [ ]:
ds = dataset_categ(ds,kategorienliste)

# Funktionen für die Kategoriendurchschnittsdarstellung

## Darstellung für die einzelnen Graphen

In [ ]:
def plot_categs(ds,i,index_selected,ax=None):
    df_befund,df_spec,df_chemshift,_,_ = ds.dataset.load_dataset(i)

    # Welche Kategorien liegen in der Auswahl vor?
    index_temp = kategorienliste.columns.isin(df_befund['Diagnosekategorie'].loc[index_selected].unique())
    categ_selec = list(compress(kategorienliste.columns.values,index_temp))

    # Wähle die richtigen Farben für die Darstellung aus
    color_selec_list = list(compress(CATEG_PALETTE,index_temp))

    # Interpoliere für einheitliche x-Werte
    new_x = new_x_values(df_chemshift)
    df_spec_interpol = interpoliere_spektren(df_spec,df_chemshift,new_x)

    if not ax:
        _, ax = plt.subplots()

    for i,categ in enumerate(categ_selec):
        index_temp = df_befund['Diagnosekategorie'].loc[index_selected]==categ
        index_temp = index_temp[index_temp].index
        avg_spec = df_spec_interpol.loc[index_temp].mean(axis=0)
        std_spec = df_spec_interpol.loc[index_temp].std(axis=0)
        color_selec = color_selec_list[i]

        ax = sns.lineplot(avg_spec,linewidth=.5,ax=ax,color=color_selec)
        ax.fill_between(new_x,avg_spec-std_spec*.2,avg_spec+std_spec*.2,alpha=0.25,linewidth=.25,color=color_selec)


    return ax

## Gesamtdarstellung

In [ ]:
def kategorien_durchschnitt(ds, vp, i=1, fig = None):
    # Wähle die richtige Befundliste aus
    df_befund,_,_,_,_ = ds.dataset.load_dataset(i)

    # Container für die gesammelten Indices
    index_container = pd.Index([])

    alter_categ_mask = ALTER_SELEC_DIAGKATEG[(i,vp)]
    alter_categ_selec = list(compress(ALTERSGRUPPE_NAMEN,alter_categ_mask)) # Welche Altersgruppen werden betrachtet
    row_n = sum(alter_categ_mask) # es gibt soviel Reihen, wie Altersgruppen betrachtet werden

    if not fig: # wenn Abbildung nicht angegeben: kreiere Abbildung
        fig = plt.figure(figsize=(20, (10/3)*row_n))
        fig.suptitle('Voxelposition: '+vp+' - '+ FILE_NAMES[i-1].lstrip('*').split('_')[0]+ ' Echozeit',size=40) #Zero-Indexing

    outer = gridspec.GridSpec(row_n, 1, wspace=0.2, hspace=.25) #kreiere Spacing der Reihen

    for j, alter_selec in enumerate(alter_categ_selec): # durchlaufe alle Altersgruppen
        index_selected = spec_selec_dataset(ds,alter_selec,vp,i=i) #Indices alle in diesem Durchgang des Loops betrachteten Spektren
        # es wird ein Datensatz, eine Voxelposition und eine Altersgruppe ausgewählt

        index_selected = entferne_nbekannt(ds,i,index_selected) # entferne die Diagnosekategorie 'Nicht bekannt'

        # wenn es keine Spektren gibt: nächster Durchgang
        if index_selected.empty:
            continue

        count = df_befund['Diagnosekategorie'].loc[index_selected].value_counts() #zähle wieviel Spektren der
                # einzelnen Kategorien enthalten sind

        if len(count)>=5:
            index_selected = df_befund['Diagnosekategorie'].loc[index_selected].isin(count[range(5)].index)
            index_selected = index_selected[index_selected].index

        # füge die Indices dem Container zu
        index_container = index_container.append(index_selected)

        inner = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=outer[j], wspace=0.1, hspace=.1) #wähle die erste Reihe aus und
        # spalte sie in zwei Spalten

        for k in range(2):
            abb_n = (j*2)+k # Abbildungsnummer, zero-indexed
            if k == 0: #eigentliche Graphendarstellung, erste Spalte

                ax = plt.Subplot(fig, inner[k])
                ax = fig.add_subplot(ax)
                ax = axis_config_categ(ax, index_selected,abb_n)
                ax = plot_categs(ds,i,index_selected,ax=ax)

            if k == 1: # Darstellung der Altersverteilung
                ax = plt.Subplot(fig, inner[k])
                ax = fig.add_subplot(ax)
                
                ax = categ_histogram(count,kategorienliste,ax=ax)
                
    return fig, index_container

# Altershistogramme für die unterschiedlichen Parameterkombinationen
*(in der Arbeit nicht verwendet)*

In [ ]:
def alter_histogramm_vorauswahl(ds, index_selec, vp, i=1, ax=None):
    df_befund,_,_,_,_ = ds.dataset.load_dataset(i)

    n = len(index_selec) #Gesamtzahl der betrachteten Spektren

    alter_auswahl = df_befund['Alter']['Jahre'].loc[index_selec]

    if not ax:
        fig = plt.figure(figsize=(10, 10))
        fig.suptitle(vp)
        ax = fig.add_subplot(1,1,1)

    bins = np.arange(start=0,stop=90,step=2)
    counts, _ = np.histogram(alter_auswahl,bins)
    sns.histplot(alter_auswahl, color=STD_COLOR, bins=bins, kde=False, ax=ax)

    # Fehlerbalken
    mean = np.mean(alter_auswahl)
    std = np.std(alter_auswahl)
    ax.errorbar(mean, np.max(counts)*1.1 , xerr=std, fmt='o', color='black', capsize=3, capthick=1)

    # Kofiguration der Achsen
    ax.set_ylim(top=np.max(counts)*1.2) # dadurch hat der Fehlerbalken auch noch Platz
    ax.set_ylabel('Anzahl der Spektren')
    ax.set_xticks(np.arange(start=0,stop=90,step=2))

    for label in ax.get_xticklabels()[::2]: #zu dichte Achsenbeschriftung anpassen
        label.set_visible(False)

    ax.text(0.95, 0.8, f'n = {n}', transform=ax.transAxes, 
                fontsize=8, verticalalignment='top',horizontalalignment='center') # Gesamtzahl der Spektren
    
    ax.text(.95, 0.7, f'Ø = {mean:.1f}', transform=ax.transAxes, 
            fontsize=8, verticalalignment='top',horizontalalignment='center') # Spektrendurchschnitt
    
    return fig

# Erstellen und Speichern der Abbildungen

In [ ]:
os.chdir("path/to/save/graphs")

In [ ]:
categ_legend,_ = create_legend()

In [ ]:
categ_legend.savefig("Legende_Kategorien.jpeg")

In [ ]:
def make_dir(fname):
    directory_name = fname
    try:
        os.mkdir(directory_name)
        print(f"Directory '{directory_name}' created successfully.")
    except FileExistsError:
        print(f"Directory '{directory_name}' already exists.")

In [ ]:
def gcdn(i): # get current dataset name
    return FILE_NAMES[i-1].strip('*')

In [ ]:
for i in range(1,5):
    folder_name = 'Datensatz_'+gcdn(i)

    #erstelle und navigiere in den entsprechenden Ordner
    make_dir(folder_name)
    os.chdir(folder_name)

    for vp in [VP_NAMES[0],VP_NAMES[1]]:
        fig_spectra,index_selected = kategorien_durchschnitt(ds,vp,i)
        fig_alterhisto = alter_histogramm_vorauswahl(ds, index_selected, vp, i=i)

        # speichere die Bilder
        fig_spectra.savefig(f'Durchschnittsspektren_Kategorien_{vp}_{gcdn(i)}.jpeg')
        fig_alterhisto.savefig(f'Altershistogramm_Kategorien_{vp}_{gcdn(i)}.jpeg')

    # navigiere aus dem Ordner wieder heraus
    os.chdir('..')